In [80]:
# import libraries
from bs4 import BeautifulSoup
import requests
import numpy as np
from time import sleep
from random import randint
import pandas as pd
import datetime
from datetime import date
import re
import unicodedata


In [81]:
# Here we are calculating the age of the celebrity. We break it down into birthday, deathday, and age(up to deathday if dead)

def calculate_age(url):
  bday = None
  death_date = None
  
  #sleep(randint(1,3))
  parser = BeautifulSoup(plain_text, 'html.parser')
      
  tables = parser.findAll('table', class_ = 'dataTable')

  for table in tables:
    if table:
      even_table_rows = table.find_all('tr', class_ = 'even')
      for row in even_table_rows:
        label_td = row.find('td', class_ = 'label')
        if label_td and 'Born' in label_td.text:
          time_element = row.find('time')
          if time_element:
            bday = time_element.get('datetime')
            year,month,day = bday.split('-')

            try:
              year = int(year)
              month = int(month)
              day = int(day)
              bday = date(year,month,day)
              break # we got what we wanted, stop looping
            except (AttributeError, ValueError):
              pass # continue looping, hopefully we have a good tag with time next

      odd_table_rows = table.find_all('tr', class_ = 'odd')
      for row in odd_table_rows:
        label_td = row.find('td', class_ = 'label')
        if label_td and 'Died' in label_td.text:
          time_element = row.find('time')
          if time_element:
            death_date = time_element.get('datetime')
            d_year,d_month,d_day = death_date.split('-')
              
            try:
              d_year = int(d_year)
              d_month = int(d_month)
              d_day = int(d_day)
              death_date = date(d_year,d_month,d_day)
              break # we got what we wanted, stop looping
            except (AttributeError, ValueError):
              pass # continue looping, hopefully we have a good tag with time next    
  
  return (bday, death_date)




In [82]:
# Function for getting spouses info, married/divorced

def get_spouse_info(url):
    marriages = {}
    tables = parser.findAll('table', class_ = 'dataTable')

    for table in tables:
        if table:
            even_table_rows = table.find_all('tr', class_ = 'even')
            for row in even_table_rows:
                tds = row.find_all('td')
                if tds and 'Spouse' in tds[0].text:

                    spouse_info = tds[1].get_text()
                    spouse_info = unicodedata.normalize("NFKD", spouse_info)

                    ss = spouse_info.split("\n")
                    ss = [s.strip() for s in ss]
                    ss = [s or "\n" for s in ss]

                    spouse_info = ' '.join(ss)
                    if spouse_info[0] == '\n':
                        spouse_info = spouse_info[1:]
                    if spouse_info[-1] == '\n':
                        spouse_info = spouse_info[:-1]

                    #print(spouse_info.split("\n"))

                    for minfo in spouse_info.split("\n"):
                        minfo = ' '.join(minfo.split())
                        li = minfo.find('(')
                        ri = minfo.find(')')
                        spouse_name = minfo[0:li].strip()
                        duration = minfo[li+1:ri].strip()
                        marriages[spouse_name] = duration
                    
                      
                      
    return(marriages)

In [83]:
# function for getting number of children

def get_children_count(url):
    tables = parser.findAll('table', class_ = 'dataTable')

    children_count = 0
    for table in tables:
        if table:
          even_table_rows = table.find_all('tr', class_ = 'even')
          for row in even_table_rows:
            tds = row.find_all('td')
            if tds and 'Children' in tds[0].text:
              children_info = tds[1].text.strip()
              lst = (children_info.split("\n"))
              for e in lst:
                if e.strip():
                  children_count += 1
                  

    return(children_count)



In [84]:
#function to determine sex of celebrity

def sex_identifier(url):
    bio = parser.find('div', class_ = 'soda odd').text
    male_pronoun = 0
    female_pronoun = 0
    res = re.findall( r'\w+|[^\s\w]+', bio.lower())
    for word in res:
        if word in ['he', 'his', 'him']:
            male_pronoun += 1
        elif word in ['she', 'her', 'hers']:
            female_pronoun += 1
    if male_pronoun > female_pronoun:
        return 'Male'
    elif female_pronoun > male_pronoun:
        return 'Female'
    else:
        None  


    

In [85]:
#running a web scrape bot to get all this info on the celebrity

pages = np.arange(1, 10000, 50)
template = 'https://www.imdb.com'

names = []
sex = []
links = []
age = []
children = []
spouse_info_dict = {}


for page in pages:
  url="https://www.imdb.com/search/name/?match_all=true&start=" + str(page) + "&ref_=rlm" 
  
  page = ''
  while page == '':
    try:
        page = requests.get(url)
        break
    except:
        print("Connection refused by the server..")
        print("Let me sleep for 5 seconds")
        print("ZZzzzz...")
        time.sleep(5)
        print("Was a nice sleep, now let me continue...")
        continue
  plain_text = page.content
  #sleep(randint(1,3))
  parser = BeautifulSoup(plain_text, 'html.parser')

  headers = parser.findAll('h3', class_ = 'lister-item-header')
  for header in headers:
    if header:
      for a in header.find_all('a'):
          if a:
            name = a.get_text().strip()
            names.append(name)
            link = template + a.get('href') + '/bio?ref_=nm_ql_1'
            links.append(link)
            break
for link in links:
  plain_text = requests.get(link).content
  parser = BeautifulSoup(plain_text, 'html.parser')
  age.append(calculate_age(link))
  children.append(get_children_count(link))
  spouse_info_dict[link] = get_spouse_info(link)
  try:
    sex.append(sex_identifier(link))
  except AttributeError:
    sex.append(None)
    continue

In [86]:
#df = pd.read_csv('celeb_dataset.csv', header = None)
#df.columns = ['Name', 'URL', 'Age', 'Children']
#df

In [100]:
df = pd.DataFrame()
df['Name'] = names
df['URL'] = links
df['Sex'] = sex
df['Birthdate/Deathdate'] = age
df['Children'] = children


In [101]:
df.info()
#df.to_csv('celeb_dataset.csv', header=False, index =False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Name                 9999 non-null   object
 1   URL                  9999 non-null   object
 2   Sex                  9436 non-null   object
 3   Birthdate/Deathdate  9999 non-null   object
 4   Children             9999 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 390.7+ KB


In [102]:
#Because we got spouse info in a dictionary, we break it down into another second dataframe and we do a full merge on the URL column with the first dataframe

url = list(spouse_info_dict.keys())
spouses = list(spouse_info_dict.values())
df2 = pd.DataFrame()
df2['URL'] = url
df2['Spouse'] = spouses
df2

,URL,Spouse
0,https://www.imdb.com/name/nm0331906/bio?ref_=n...,{'Dara Gottfried': '3 February 2007 - 12 April...
1,https://www.imdb.com/name/nm1395532/bio?ref_=n...,{}
2,https://www.imdb.com/name/nm0002907/bio?ref_=n...,{}
3,https://www.imdb.com/name/nm7592760/bio?ref_=n...,{}
4,https://www.imdb.com/name/nm2443758/bio?ref_=n...,{'Brooklyn Peltz Beckham': '9 April 2022 - pre...
...,...,...
9994,https://www.imdb.com/name/nm1413858/bio?ref_=n...,{}
9995,https://www.imdb.com/name/nm0363669/bio?ref_=n...,{}
9996,https://www.imdb.com/name/nm4434808/bio?ref_=n...,{'Taige Jensen': '19 October 2019 - present'}
9997,https://www.imdb.com/name/nm0038673/bio?ref_=n...,{}


In [90]:
#just checking to see what's the most spouse a celebrity has 

maximum = 0
for spouse in spouses:
    if len(spouse) > maximum:
        maximum = len(spouse)
    else:
        pass
print(maximum)

9


In [91]:
df_final = pd.merge(df, df2, on='URL', how='outer')

In [159]:
df_final

,Name,URL,Sex,Birthdate/Deathdate,Children,Spouse,Number of marriages
0,Gilbert Gottfried,https://www.imdb.com/name/nm0331906/bio?ref_=n...,Male,"(datetime.date(1955, 2, 28), datetime.date(202...",2,{'Dara Gottfried': '3 February 2007 - 12 April...,1
1,Joseph Gatt,https://www.imdb.com/name/nm1395532/bio?ref_=n...,Male,"(datetime.date(1971, 12, 3), None)",0,{},0
2,Alexander Skarsgård,https://www.imdb.com/name/nm0002907/bio?ref_=n...,Male,"(datetime.date(1976, 8, 25), None)",0,{},0
3,Simone Ashley,https://www.imdb.com/name/nm7592760/bio?ref_=n...,Female,"(datetime.date(1995, 3, 30), None)",0,{},0
4,Nicola Peltz Beckham,https://www.imdb.com/name/nm2443758/bio?ref_=n...,Female,"(datetime.date(1995, 1, 9), None)",0,{'Brooklyn Peltz Beckham': '9 April 2022 - pre...,1
...,...,...,...,...,...,...,...
9994,Trishna,https://www.imdb.com/name/nm1413858/bio?ref_=n...,Female,"(datetime.date(1988, 12, 14), None)",0,{},0
9995,Michael Harney,https://www.imdb.com/name/nm0363669/bio?ref_=n...,Male,"(None, None)",0,{},0
9996,Jenn Lyon,https://www.imdb.com/name/nm4434808/bio?ref_=n...,Female,"(None, None)",0,{'Taige Jensen': '19 October 2019 - present'},1
9997,William Ash,https://www.imdb.com/name/nm0038673/bio?ref_=n...,Male,"(datetime.date(1977, 1, 13), None)",0,{},0


In [93]:
number_of_spouses = []
for spouse in df_final['Spouse']:
    number_of_spouses.append(len(spouse))

df_final['Number of marriages'] = number_of_spouses
df_final

,Name,URL,Sex,Birthdate/Deathdate,Children,Spouse,Number of marriages
0,Gilbert Gottfried,https://www.imdb.com/name/nm0331906/bio?ref_=n...,Male,"(1955-02-28, 2022-04-12)",2,{'Dara Gottfried': '3 February 2007 - 12 April...,1
1,Joseph Gatt,https://www.imdb.com/name/nm1395532/bio?ref_=n...,Male,"(1971-12-03, None)",0,{},0
2,Alexander Skarsgård,https://www.imdb.com/name/nm0002907/bio?ref_=n...,Male,"(1976-08-25, None)",0,{},0
3,Simone Ashley,https://www.imdb.com/name/nm7592760/bio?ref_=n...,Female,"(1995-03-30, None)",0,{},0
4,Nicola Peltz Beckham,https://www.imdb.com/name/nm2443758/bio?ref_=n...,Female,"(1995-01-09, None)",0,{'Brooklyn Peltz Beckham': '9 April 2022 - pre...,1
...,...,...,...,...,...,...,...
9994,Trishna,https://www.imdb.com/name/nm1413858/bio?ref_=n...,Female,"(1988-12-14, None)",0,{},0
9995,Michael Harney,https://www.imdb.com/name/nm0363669/bio?ref_=n...,Male,"(None, None)",0,{},0
9996,Jenn Lyon,https://www.imdb.com/name/nm4434808/bio?ref_=n...,Female,"(None, None)",0,{'Taige Jensen': '19 October 2019 - present'},1
9997,William Ash,https://www.imdb.com/name/nm0038673/bio?ref_=n...,Male,"(1977-01-13, None)",0,{},0


In [168]:
spouse_set = []
for spouse in spouses:
    spouse_set.append(list(spouse.items()))
spouse_set

[[('Dara Gottfried', '3 February 2007 - 12 April 2022')],
 [],
 [],
 [],
 [('Brooklyn Peltz Beckham', '9 April 2022 - present')],
 [('Johnny Depp', '3 February 2015 - 13 January 2017')],
 [('Elvira Lind', 'March 2017 - present')],
 [],
 [],
 [],
 [],
 [],
 [],
 [('Kate Hannan', '7 April 1962 - present')],
 [('Marc Clotet', 'July 2011 - 2013')],
 [('Riko Shibata', '16 February 2021 - present'),
  ('Alice Kim Cage', '30 July 2004 - January 2016'),
  ('Lisa Marie Presley', '10 August 2002 - 16 May 2004'),
  ('Patricia Arquette', '8 April 1995 - 18 May 2001')],
 [('Jessica Monty', '2007 - present')],
 [],
 [('Amber Heard', '3 February 2015 - 13 January 2017'),
  ('Lori A. Depp', '24 December 1983 - 7 March 1986')],
 [('Maximilian Osinski', '24 January 2015 - present')],
 [],
 [],
 [],
 [],
 [('Shia LaBeouf', '10 October 2016 - present')],
 [],
 [],
 [],
 [('Brian Calicchia', '2015 - present'),
  ('Bradley Lieberman', '14 August 2010 - 2014')],
 [],
 [('Dickson Poon', 'February 1988 - 1992'

In [175]:
married_list = []
for i in range(len(spouse_set)):
    try:
        if spouse_set[i][0][-1].endswith('present'):
            married_list.append(1)
        else:
            married_list.append(0)
    except IndexError:
        married_list.append(0)

In [177]:
df_final['Spouse'] = spouse_set

In [178]:
df_final['Married?'] = married_list
df_final

,Name,URL,Sex,Birthdate/Deathdate,Children,Spouse,Number of marriages,Married?
0,Gilbert Gottfried,https://www.imdb.com/name/nm0331906/bio?ref_=n...,Male,"(datetime.date(1955, 2, 28), datetime.date(202...",2,"[(Dara Gottfried, 3 February 2007 - 12 April 2...",1,0
1,Joseph Gatt,https://www.imdb.com/name/nm1395532/bio?ref_=n...,Male,"(datetime.date(1971, 12, 3), None)",0,[],0,0
2,Alexander Skarsgård,https://www.imdb.com/name/nm0002907/bio?ref_=n...,Male,"(datetime.date(1976, 8, 25), None)",0,[],0,0
3,Simone Ashley,https://www.imdb.com/name/nm7592760/bio?ref_=n...,Female,"(datetime.date(1995, 3, 30), None)",0,[],0,0
4,Nicola Peltz Beckham,https://www.imdb.com/name/nm2443758/bio?ref_=n...,Female,"(datetime.date(1995, 1, 9), None)",0,"[(Brooklyn Peltz Beckham, 9 April 2022 - prese...",1,1
...,...,...,...,...,...,...,...,...
9994,Trishna,https://www.imdb.com/name/nm1413858/bio?ref_=n...,Female,"(datetime.date(1988, 12, 14), None)",0,[],0,0
9995,Michael Harney,https://www.imdb.com/name/nm0363669/bio?ref_=n...,Male,"(None, None)",0,[],0,0
9996,Jenn Lyon,https://www.imdb.com/name/nm4434808/bio?ref_=n...,Female,"(None, None)",0,"[(Taige Jensen, 19 October 2019 - present)]",1,1
9997,William Ash,https://www.imdb.com/name/nm0038673/bio?ref_=n...,Male,"(datetime.date(1977, 1, 13), None)",0,[],0,0
